In [1]:
using Flight.FlightCore.Systems
using Flight.FlightCore.Plotting

using Flight.FlightPhysics.Geodesy
using Flight.FlightPhysics.Kinematics
using Flight.FlightComponents.Aircraft
using Flight.FlightComponents.Control
using Flight.FlightAircraft.C172FBW
using Flight.FlightAircraft.C172

using UnPack
using ControlSystems
using RobustAndOptimalControl
using ComponentArrays
using LinearAlgebra

In [2]:
ac = Cessna172FBWBase(NED()) |> System #linearization requires NED kinematics

#same mass in all cases
fwd_cg_pld = C172.PayloadU(m_pilot = 100, m_copilot = 100, m_baggage = 0)
aft_cg_pld = C172.PayloadU(m_pilot = 50, m_copilot = 50, m_baggage = 100)
mid_cg_pld = C172.PayloadU(m_pilot = 75, m_copilot = 75, m_baggage = 50)

design_condition = C172.TrimParameters(
    Ob = Geographic(LatLon(), HOrth(1000)),
    EAS = 40.0,
    γ_wOb_n = 0.0,
    x_fuel = 0.5,
    flaps = 0.0,
    payload = mid_cg_pld)

lss_lon = Control.LinearStateSpace(ac, design_condition; model = :lon);
nss_lon = named_ss(lss_lon);

x_labels = [:q, :θ, :v_x, :v_z, :α_filt, :ω_eng, :ele_v, :ele_p, :thr_v, :thr_p]
u_labels = [:elevator_cmd, :throttle_cmd]
y_labels = [:q, :θ, :α, :EAS, :TAS, :f_x, :f_z, :γ, :c, :ω_eng, :v_D]
# x_labels_pitch = [:q, :θ, :v_x, :v_z, :α_filt, :ele_v, :ele_p]
# u_labels_pitch = [:elevator_cmd]
# y_labels_pitch = [:q, :θ, :α, :EAS]

lss = submodel(lss_lon; x = x_labels, u = u_labels, y = y_labels)
nss = named_ss(lss);

## 1. Longitudinal Dynamics Regulator

In [3]:
dampreport(nss_lon)
dampreport(nss)

|        Pole        |   Damping     |   Frequency   |   Frequency   | Time Constant |
|                    |    Ratio      |   (rad/sec)   |     (Hz)      |     (sec)     |
+--------------------+---------------+---------------+---------------+---------------+


| -0.000369          |  1            |  0.000369     |  5.87e-05     |  2.71e+03     |


| -0.0176 ±  0.293im |  0.0601       |  0.293        |  0.0467       |  56.8         |
| -4.42   ±   5.14im |  0.652        |  6.78         |  1.08         |  0.226        |
| -9.72              |  1            |  9.72         |  1.55         |  0.103        |
| -18.8   ±   25.1im |  0.6          |  31.4         |  5            |  0.0531       |
| -18.8   ±   25.1im |  0.6          |  31.4         |  5            |  0.0531       |
| -48.1              |  1            |  48.1         |  7.65         |  0.0208       |
|        Pole        |   Damping     |   Frequency   |   Frequency   | Time Constant |
|                    |    Ratio      |   (rad/sec)   |     (Hz)      |     (sec)     |
+--------------------+---------------+---------------+---------------+---------------+
| -0.0178 ±  0.291im |  0.061        |  0.292        |  0.0465       |  56.2         |
| -4.42   ±   5.14im |  0.652        |  6.78         |  1.08         |  0.226        |
| -9.72              |  1            |  9.7

We see that removing $h$ as a state eliminates an extremely slow pole, probably related to the effect of altitude on engine output. The remaining poles are virtually unaffected. Also, it significantly improves the condition of the dynamics matrix:

In [4]:
cond(nss_lon.A) |> display
cond(nss.A) |> display

7.656238481914206e7

440485.8084271008

Thus, we will take the MIMO system without $h$ as our plant baseline. Let's start by designing a full-state feedback regulator for the MIMO system.

In [5]:
gram(nss, :c)

10×10 Matrix{Float64}:
   5.24175       1.73723e-6     16.7863   …      -1.50383       0.0315657
   1.73723e-6    2.18367       -11.5022          -0.0315657    -0.000317977
  16.7863      -11.5022       2237.85            -8.17028       0.13902
  10.7133        6.44921        12.2094          -1.2785       -0.00806454
   0.131826      0.175557       -3.5167          -0.00366141   -0.000503233
  61.871       -19.3129       5345.69     …  -10227.3         490.683
 -93.4314       -4.34506        77.9313           0.0           0.0
   4.34507       0.0713036      -3.16453          0.0           0.0
  -1.50383      -0.0315657      -8.17028      12919.3           0.0
   0.0315657    -0.000317977     0.13902          0.0          13.09

The open-loop system is:
$$\Delta \dot{x} = A \Delta x + B \Delta u$$
$$\Delta y = C \Delta x$$

The regulator control law is simply $\Delta u = -K \Delta x$, which results in a homogeneous closed-loop system:
$$\Delta \dot{x} = (A -BK) \Delta x$$
$$\Delta y = C \Delta x$$

To retain control over the system, we instead use the control law $\Delta u = -K \Delta x + \Delta u_{ext}$, which gives:
$$\Delta \dot{x} = (A - BK) \Delta x + B \Delta u_{ext}$$
$$\Delta y = C \Delta x$$

This system has the same stability properties as the homogeneous one.

## FROM HERE. Scale v_x and v_z costs with v norm

In [6]:
#we do not particularly care about driving the elevator velocity and position
#back to zero as fast as possible. we do care about q, θ and α
v_ref = sqrt(lss.x0.v_x^2 + lss.x0.v_z^2)
diagQ_pitch = ComponentVector(q = 1, θ = 1, v_x = 1, v_z = 1, α_filt = 0, ele_v = 0, ele_p = 0)
Q_pitch = ComponentMatrix(diagm(diagQ_pitch), Axis(x_labels_pitch), Axis(x_labels_pitch))
R_pitch = 5

#gain matrix for the reduced pitch dynamics
K_pitch = lqr(nss_pitch, Q_pitch, R_pitch)
display(K_pitch)

#add pitch regulation to the reduced pitch dynamics
A_pitch_sas = lss_pitch.A - lss_pitch.B * K_pitch
nss_pitch_sas = named_ss(ss(A_pitch_sas, lss_pitch.B, lss_pitch.C, 0), u = [:sas_pitch_cmd], x = nss_pitch.x, y = nss_pitch.y);

dampreport(nss_pitch)
dampreport(nss_pitch_sas)
e2q = nss_pitch[:q, :elevator_cmd]
e2q_sas = nss_pitch_sas[:q, :sas_pitch_cmd]
e2θ = nss_pitch[:θ, :elevator_cmd]
e2θ_sas = nss_pitch_sas[:θ, :sas_pitch_cmd]
step(e2q, 10) |> plot
step(e2q_sas, 10) |> plot!

A_pitch_sas[:θ, :]

UndefVarError: UndefVarError: `x_labels_pitch` not defined

In [7]:
#build a gain matrix for the complete longitudinal dynamics and assign the
#components corresponding to the gain matrix computed from the reduced pitch
#dynamics
K_pitch_full = fill!(lss_lon.u0 * lss_lon.x0', 0)
K_pitch_full[:elevator_cmd, x_labels_pitch] = K_pitch
A_pitch_full_sas = lss_lon.A - lss_lon.B * K_pitch_full;

#add pitch regulation to the full longitudinal dynamics
nss_lon_sas = named_ss(ss(A_pitch_full_sas, lss_lon.B, lss_lon.C, 0), u = [:sas_pitch_cmd, :throttle_cmd], x = nss_lon.x, y = nss_lon.y);

dampreport(nss_pitch)
dampreport(nss_lon_sas)
e2q = nss_lon[:q, :elevator_cmd]
e2q_sas = nss_lon_sas[:q, :sas_pitch_cmd]
e2θ = nss_lon[:θ, :elevator_cmd]
e2θ_sas = nss_lon_sas[:θ, :sas_pitch_cmd]
step(e2q, 10) |> plot
step(e2q_sas, 10) |> plot!

UndefVarError: UndefVarError: `K_pitch` not defined

In [8]:
bodeplot(e2q)

UndefVarError: UndefVarError: `e2q` not defined

In [9]:
bodeplot(e2q_sas)

UndefVarError: UndefVarError: `e2q_sas` not defined

In [10]:

P_e2q = e2q_sas
C_q2e = named_ss(ss(20 * tf(1, [1, 0, 0])), :C_q2e; u = :q_err, y = :sas_pitch_cmd);
L_q2e = series(C_q2e, P_e2q)
T_q2e = output_comp_sensitivity(P_e2q, C_q2e)
# CS_q2e = G_CS(P_e2q, C_q2e)

marginplot(L_q2e)
# marginplot(CS_q2e)
step(T_q2e, 10) |> plot

UndefVarError: UndefVarError: `e2q_sas` not defined